<a href="https://colab.research.google.com/github/kangwonlee/nmisp/blob/main/50_ode/60_Duffing_Oscillator.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


In [ ]:
# This cell is for the Google Colaboratory
# https://stackoverflow.com/a/63519730
if 'google.colab' in str(get_ipython()):
  path_py = '/content/nmisp_py'

  import os
  if not os.path.exists(path_py):
    import subprocess
    subprocess.run(
        ('git', 'clone', 'https://github.com/kangwonlee/nmisp_py')
    )
  assert os.path.exists(path_py)

  import sys
  sys.path.insert(0, path_py)



In [ ]:
import os

import matplotlib.pyplot as plt
import matplotlib.gridspec as mg
import numpy as np



In [ ]:
import ode_solver



# Duffing Oscillator<br>더핑 진동자



The Duffing oscillator is a simple model of a mass-spring system with a nonlinear spring. The nonlinearity arises from the fact that the spring's restoring force is not proportional to its displacement. The Duffing oscillator is described by the following second-order differential equation:



$$
m\frac{d^2}{dt^2}x(t) + c\frac{d}{dt}x(t) + k x(t) + \alpha x^3 = F cos \omega t
$$


| symbol<br>기호 | unit<br>단위 | description<br>설명 |
|:-------------:|:-------------:|-------------|
|  $t$ | sec | time |
|  $x(t)$ | m | the displacement of the mass from its equilibrium position |
|  $\frac{d}{dt}x(t)$ | m/s | the velocity of the mass. |
|  $\frac{d^2}{dt^2}x(t)$ | m/s/s | the acceleration of the mass. |
|  $m$ | kg | the mass of the object attached to the spring. |
|  $c$ | N/(m/s) | the damping coefficient. |
|  $k$ | N/m | the linear stiffness coefficient of the spring. |
|  $\alpha$ | $N/{m^3}$ | the nonlinear stiffness coefficient of the spring. |
|  $F$ | N | the amplitude of the external driving force. |
|  $\omega$ | rad/sec | the angular frequency of the external driving force. |



## The Nonlinear Term

The key feature of the Duffing equation is the term $\alpha x^3$. This term introduces a cubic nonlinearity into the system.  When the displacement $x$ is small, this term is negligible, and the system behaves like a simple harmonic oscillator. However, as the displacement increases, the nonlinear term becomes significant, leading to deviations from simple harmonic motion.

## Physical Interpretation

The Duffing oscillator can model a variety of physical systems, including:

* Mechanical Systems: A mass-spring system where the spring's stiffness changes as it is stretched or compressed.
* Electrical Circuits: An RLC circuit with a nonlinear inductor or capacitor.
* Structural Systems: A beam or plate with large deflections, where the geometric nonlinearities become important.

## Chaotic Behavior

For certain combinations of parameters (m, c, k, alpha, F, omega), the Duffing oscillator can exhibit chaotic behavior. This means that the system's response is extremely sensitive to initial conditions, and its long-term behavior is unpredictable.

## Numerical Solution

The Duffing equation is a nonlinear differential equation, and in most cases, it does not have an analytical solution. Therefore, numerical methods, such as the Runge-Kutta methods, are essential for simulating the behavior of the Duffing oscillator.


In [ ]:
m_kg = 1.0
c_Npmps = 0.25
k_Npm = 1
alpha_Npm3 = 0.1
F_N = 0.5
omega_rps = 1.5



In [ ]:
def get_Duffing_oscillator_slope(
        m_kg:float, c_Npmps:float, k_Npm:float, alpha_Npm3:float,
        F_N:float, omega_rps:float):
    '''
    Return a closure calculating slope function
    기울기 함수를 계산하는 내포 함수를 반환
    '''

    neg_m_inv = (-1.0) / m_kg
    neg_c_m = c_Npmps * neg_m_inv
    neg_k_m = k_Npm * neg_m_inv
    neg_alpha_m = alpha_Npm3 * neg_m_inv
    F_m = F_N / m_kg

    def f(t:float, xv:np.ndarray,):
        '''
        Calculate slope vector of the Duffing Oscillator and a linear oscillator
        더핑 진동자와 선형 진동계의 기울기 벡터를 계산
        '''
        x, v, x2, v2 = xv
        dx_dt = v
        dv_dt = neg_k_m * x + neg_c_m * v + neg_alpha_m * x**3 + F_m * np.cos(omega_rps * t)
        dx2_dt = v2
        dv2_dt = neg_k_m * x2 + neg_c_m * v2 + F_m * np.cos(omega_rps * t)

        return np.array((dx_dt, dv_dt, dx2_dt, dv2_dt))

    return f



In [ ]:
def run_sim(
        slope, x_0:np.ndarray=np.array((0.0, 0.0, 0.0, 0.0)),
        t_start=0.0, t_end=120.0, delta_t_sec=1e-3,
    ):
    '''
    Simulate a one degree of freedom vibration system
    1자유도 진동계 시뮬레이션
    '''

    if os.getenv('CI', False):
        t_end = t_start + 1.0

    t_array = np.arange(t_start, t_end, delta_t_sec)

    t_out, x_out = ode_solver.rk4(slope, t_array, x_0)
    x_out = np.array(x_out)
    return t_out, x_out


def sim_1dof_vib(
        slope, x_0:np.ndarray=np.array((0.0, 0.0, 0.0, 0.0)),
        t_start=0.0, t_end=120.0, figsize=(14, 14),
        delta_t_sec=1e-3,
    ):

    t_out, x_out = run_sim(
        slope, x_0,
        t_start, t_end, delta_t_sec,
    )

    fig = plt.figure(figsize=figsize)
    gs = mg.GridSpec(2, 2, height_ratios=[1, 1], width_ratios=[1, 1])

    axs = (
        fig.add_subplot(gs[0, 0]),
        fig.add_subplot(gs[1, 0]),
        fig.add_subplot(gs[:, 1])
    )

    for k in range(2):
        axs[k].plot(t_out, x_out[:, k], label='Duffing')
        axs[k].plot(t_out, x_out[:, k+2], label='Linear')
        axs[k].grid(True)
        axs[k].set_xlabel('t(sec)')
    axs[0].set_ylabel('$x(t)$')
    axs[1].set_ylabel(r'$\frac{d}{dt}x(t)$')

    axs[2].plot(x_out[:, 0], x_out[:, 1], label='Duffing')
    axs[2].plot(x_out[:, 2], x_out[:, 3], label='Linear')
    axs[2].grid(True)
    axs[2].set_xlabel('$x(t)$')
    axs[2].set_ylabel(r'$\frac{d}{dt}x(t)$')

    return axs



* Weak forcing near the natural frequency.



In [ ]:
F_N, omega_rps = 0.1, 1

Duffing_oscillator = get_Duffing_oscillator_slope(m_kg, c_Npmps, k_Npm, alpha_Npm3, F_N, omega_rps)
ax = sim_1dof_vib(Duffing_oscillator,)



* Stronger forcing, potentially leading to nonlinear effects.



In [ ]:
F_N, omega_rps = 0.5, 1.5

Duffing_oscillator = get_Duffing_oscillator_slope(m_kg, c_Npmps, k_Npm, alpha_Npm3, F_N, omega_rps)
ax = sim_1dof_vib(Duffing_oscillator,)



* Exploring different resonance regions.



In [ ]:
F_N, omega_rps = 1.0, 0.5

Duffing_oscillator = get_Duffing_oscillator_slope(m_kg, c_Npmps, k_Npm, alpha_Npm3, F_N, omega_rps)
ax = sim_1dof_vib(Duffing_oscillator,)



In [ ]:
F_N, omega_rps = 1.0, 1.0

Duffing_oscillator = get_Duffing_oscillator_slope(m_kg, c_Npmps, k_Npm, alpha_Npm3, F_N, omega_rps)
ax = sim_1dof_vib(Duffing_oscillator,)



In [ ]:
F_N, omega_rps = 1.0, 2.0

Duffing_oscillator = get_Duffing_oscillator_slope(m_kg, c_Npmps, k_Npm, alpha_Npm3, F_N, omega_rps)
ax = sim_1dof_vib(Duffing_oscillator,)



* Sweeping omega to look for period-doubling bifurcations and chaos.



In [ ]:
F_N, omega_rps = 1.5, 0.8

Duffing_oscillator = get_Duffing_oscillator_slope(m_kg, c_Npmps, k_Npm, alpha_Npm3, F_N, omega_rps)
ax = sim_1dof_vib(Duffing_oscillator,)



In [ ]:
F_N, omega_rps = 1.5, 1.2

Duffing_oscillator = get_Duffing_oscillator_slope(m_kg, c_Npmps, k_Npm, alpha_Npm3, F_N, omega_rps)
ax = sim_1dof_vib(Duffing_oscillator,)



* Strong forcing showing nonlinear force effects



In [ ]:
F_N, omega_rps = 2.5, 0.48

Duffing_oscillator = get_Duffing_oscillator_slope(m_kg, c_Npmps, k_Npm, alpha_Npm3, F_N, omega_rps)
ax = sim_1dof_vib(Duffing_oscillator,)



In [ ]:
import multiprocessing as mp
import scipy.optimize as so


def cost(x):
    delta_t_sec = 1e-3
    period_sec = (k_Npm / m_kg) ** 0.5
    n_fold = int(period_sec / delta_t_sec)

    F_N, omega_rps = x

    Duffing_oscillator = get_Duffing_oscillator_slope(m_kg, c_Npmps, k_Npm, alpha_Npm3, F_N, omega_rps)
    t, x = run_sim(
        Duffing_oscillator, x_0=np.array((0.0, 0.0, 0.0, 0.0)),
        t_start=0.0, t_end=120.0, delta_t_sec=1e-3,
    )

    # after transient response
    i_start = n_fold * 30
    x_sample = x[i_start:, 0].squeeze()
    x_period = x_sample.reshape(int(n_fold), -1)
    x_top = x_period.max(axis=0)
    return (- x_top.std())



In [ ]:
result = so.minimize(cost, (1, 1))
result

